# ANALYZING NEIGHBOURHOODS OF TORONTO (K-means)

In [233]:
#importing libraries required for this project
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

In [234]:
#scraping data from wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source,'lxml')
table_data = soup.find('table')
tags = table_data.find_all('tr')
postcode=[]
borough=[]
neighbourhood =[]
for i in tags:
    indi = i.find_all('td')
    if len(indi)!=0:
        if (indi[1].text != 'Not assigned'):
            postcode.append(indi[0].text)
            borough.append(indi[1].text)
            if (indi[2].text.replace('\n',"") == 'Not assigned'):
                neighbourhood.append(indi[1].text)
            else: 
                neighbourhood.append(indi[2].text.replace('\n',""))

#creating a dataframe 
data = {'Postcode':postcode, 'Borough':borough, 'Neighbourhood':neighbourhood}
df = pd.DataFrame(data=data)
df = df[['Postcode','Borough','Neighbourhood']]

#grouping the dataframe by Postcode and Borough
df = df.groupby(['Postcode','Borough']).agg(lambda x: ", ".join(x.astype(str))).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [235]:
#shape of the dataframe
print(df.shape)

(103, 3)


In [236]:
# #getting the latitude and longitude corresponding to each postcode
#TAKES A LOT OF TIME DUE TO SLOW RESPONSE TIME OF GEOCODER
# latitude=[]
# longitude=[]
# for postal_code in df['Postcode']:
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#       lat_lng_coords = g.latlng
#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])
# df.insert(len(df.columns),'Latitude',latitude)
# df.insert(len(df.columns),'Longitude',longitude)
# df.head()

In [238]:
!wget http://cocl.us/Geospatial_data
print("Data Downloaded!")
df_geo = pd.read_csv("Geospatial_data")
df_geo.head()

Data Downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [239]:
df_geo.reset_index()
df_geo.columns = ['Postcode','Latitude','Longitude']

#joining two dataframes on the Postcode column 
df_new = pd.merge(df,df_geo,on = 'Postcode')
df_new.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [240]:
#defining the Four_square API credentials
CLIENT_ID = 'WWG5PPSIN5WMPCBSTOSVZWVL5L1UZY0UJTNSVIZBWFIRA22V' 
CLIENT_SECRET = 'ZUJ3ZJIWV1M4C5T0XWSZSVGECWH42YKEHNIQLMP5ZZYJ350G' 
VERSION = '20180605'
LIMIT = 100
radius = 500

In [241]:
venues =[]
for name,lat,lng in zip(df_new['Neighbourhood'],df_new['Latitude'],df_new['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']
    for i in results:
        venues.append([
            name, 
            lat, 
            lng, 
            i['venue']['name'], 
            i['venue']['location']['lat'], 
            i['venue']['location']['lng'],  
            i['venue']['categories'][0]['name']])        

In [242]:
nearby_venues =pd.DataFrame.from_records(venues)
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
nearby_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [243]:
nearby_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",10,10,10,10,10,10


In [244]:
print("There are a {} different unique venue categories".format(len(nearby_venues['Venue Category'].unique())))

There are a 274 different unique venue categories


In [245]:
# using one hot encoding for the different venue categories
venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

#adding the Neighborhoods column from the nearby_venues dataframe
venues_onehot['Neighborhood'] = nearby_venues['Neighborhood']

#reordering the venues_onehot dataframe to have Neighborhoods as the first column
list_columns = venues_onehot.columns.tolist()
list_columns.remove('Neighborhood')
new_list=['Neighborhood']
new_list.extend(list_columns)
venues_onehot = venues_onehot[new_list]
venues_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [246]:
venues_onehot_grouped = venues_onehot.groupby(['Neighborhood']).mean().reset_index()
venues_onehot_grouped.head()
# venues_onehot_grouped.shape

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Agincourt,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [248]:
#printing top 5 most common venues for each neighbourhood
for ngh in venues_onehot_grouped['Neighborhood']:
#     print("----"+ngh+"----")
    temp = venues_onehot_grouped[venues_onehot_grouped['Neighborhood'] == ngh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(5))
#     print("\n")

As we can see, when we select the top 5 most common venues for each neighborhood, few most common places in a few neighbourhoods have a mean value of 0.0 indicating that the neighborhood does even have 5 common places. We are not interested in such neighborhoods and hence lets drop these rows from our venues_onehot_grouped dataframe.

In [249]:
venues_onehot_grouped_sum = venues_onehot.groupby(['Neighborhood']).sum().reset_index()
venues_sum = venues_onehot_grouped_sum.sum(axis=1)
venues_onehot_grouped_sum = venues_onehot_grouped_sum.loc[venues_sum >= 5,:]
venues_onehot_grouped_sum.reset_index(drop = True)
venues_onehot_grouped.set_index('Neighborhood',inplace=True)
venues_onehot_grouped = venues_onehot_grouped.loc[venues_onehot_grouped_sum['Neighborhood'],:]

In [250]:
venues_onehot_grouped.reset_index(inplace=True)

In [251]:
#function to return num_top_venues most common venues for a neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [252]:
#storing the top 5 venues corresponding to each neighborhood in a datframe
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_onehot_grouped['Neighborhood']

for ind in np.arange(venues_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Café,Steakhouse,Thai Restaurant
1,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Chinese Restaurant
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Liquor Store,Japanese Restaurant,Sandwich Place
3,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy
4,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Bank,Deli / Bodega,Restaurant


In [253]:
#performing K-means clustering on the venues_onehot_grouped dataframe
clustering_data = venues_onehot_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = 2,random_state=0).fit(clustering_data)
kmeans.labels_

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [254]:
df_new.set_index('Neighbourhood',inplace=True)
final_df=df_new.loc[venues_onehot_grouped['Neighborhood'],:]
final_df.head()

,Postcode,Borough,Latitude,Longitude
Neighbourhood,,,,
"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568
Agincourt,M1S,Scarborough,43.794200,-79.262029
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",M9V,Etobicoke,43.739416,-79.588437
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484
"Bathurst Manor, Downsview North, Wilson Heights",M3H,North York,43.754328,-79.442259


In [255]:
final_df.reset_index(inplace=True)
final_df.columns=['Neighborhood','Postcode','Borough','Latitude','Longitude']

In [256]:
final_df['labels'] = kmeans.labels_
merged_df = pd.merge(final_df,neighborhoods_venues_sorted,on='Neighborhood')
merged_df.head()

,Neighborhood,Postcode,Borough,Latitude,Longitude,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568,0,Coffee Shop,Bar,Café,Steakhouse,Thai Restaurant
1,Agincourt,M1S,Scarborough,43.794200,-79.262029,1,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Chinese Restaurant
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V,Etobicoke,43.739416,-79.588437,1,Grocery Store,Pharmacy,Liquor Store,Japanese Restaurant,Sandwich Place
3,"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,1,Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy
4,"Bathurst Manor, Downsview North, Wilson Heights",M3H,North York,43.754328,-79.442259,1,Coffee Shop,Pizza Place,Bank,Deli / Bodega,Restaurant


In [257]:
#fetching the coordinates of Toronto using the geopy library
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [258]:
#visualising the above clusters using folium on the map of toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(2)
colors_array = cm.rainbow(np.linspace(0, 1, len(x)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# FINAL CONCLUSIONS

Let us now make a few conclusions as to how the clusters might have been formed. Lets look at the at two cluster labels individually.

In [259]:
cluster_0 = merged_df.loc[merged_df['labels']==0,:]
cluster_0

,Neighborhood,Postcode,Borough,Latitude,Longitude,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568,0,Coffee Shop,Bar,Café,Steakhouse,Thai Restaurant
6,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Steakhouse
8,"Brockton, Exhibition Place, Parkdale Village",M6K,West Toronto,43.636847,-79.428191,0,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store
11,"Cabbagetown, St. James Town",M4X,Downtown Toronto,43.667967,-79.367675,0,Coffee Shop,Restaurant,Park,Pizza Place,Bakery
13,Canada Post Gateway Processing Centre,M7R,Mississauga,43.636966,-79.615819,0,Hotel,Coffee Shop,Gym / Fitness Center,Middle Eastern Restaurant,Fried Chicken Joint
14,Cedarbrae,M1H,Scarborough,43.773136,-79.239476,0,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant
15,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar
16,"Chinatown, Grange Park, Kensington Market",M5T,Downtown Toronto,43.653206,-79.400049,0,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Bakery
17,Christie,M6G,Downtown Toronto,43.669542,-79.422564,0,Café,Grocery Store,Park,Italian Restaurant,Baby Store
18,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Burger Joint,Restaurant


In [260]:
cluster_1 = merged_df.loc[merged_df['labels']==1,:]
cluster_1

,Neighborhood,Postcode,Borough,Latitude,Longitude,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Agincourt,M1S,Scarborough,43.794200,-79.262029,1,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Chinese Restaurant
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V,Etobicoke,43.739416,-79.588437,1,Grocery Store,Pharmacy,Liquor Store,Japanese Restaurant,Sandwich Place
3,"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,1,Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy
4,"Bathurst Manor, Downsview North, Wilson Heights",M3H,North York,43.754328,-79.442259,1,Coffee Shop,Pizza Place,Bank,Deli / Bodega,Restaurant
5,"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750,1,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Pharmacy,Greek Restaurant
7,"Bloordale Gardens, Eringate, Markland Wood, Ol...",M9C,Etobicoke,43.643515,-79.577201,1,Pizza Place,Convenience Store,Pharmacy,Liquor Store,Beer Store
9,Business Reply Mail Processing Centre 969 Eastern,M7Y,East Toronto,43.662744,-79.321558,1,Pizza Place,Garden Center,Brewery,Skate Park,Park
10,"CN Tower, Bathurst Quay, Island airport, Harbo...",M5V,Downtown Toronto,43.628947,-79.394420,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boutique
12,Caledonia-Fairbanks,M6E,York,43.689026,-79.453512,1,Park,Women's Store,Pharmacy,Fast Food Restaurant,Market
20,"Clarks Corners, Sullivan, Tam O'Shanter",M1T,Scarborough,43.781638,-79.304302,1,Pizza Place,Noodle House,Italian Restaurant,Pharmacy,Fast Food Restaurant


As seen from the above dataframes corresponding to each cluster label, the following conclusions can be made:
1. Label 0 clusters mainly consists of Neighborhoods whose top 5 most common venues have a Coffee Shop or Cafe.
2. Label 1 clusters mainly consists of Neighborhoods whose top 5 most common venues have a Pizza Place, Sandwich place or a Skating rink.

This information can be used a variety of ways. Lets say we are a network provider company like T mobile. We can give discount coupons depending on the most common places in a particular neighborhood. We can provide Coffee Shop or Cafe discount coupons to label 0 neighborhoods and discount coupons for Pizza Places, Sandwich places or a Skating rinks for label 1 clusters which might increase the revenue of the company. 